In [89]:
import pandas as pd
import fundingutils
import plotly.graph_objs as go
import plotly.express as px
import numpy as np


In [90]:
votes = pd.read_csv('data/gg19_votes.csv')
votes.rename(columns={'grantAddress': 'payoutAddress'}, inplace=True)
# Display summary statistics
print("Number of unique rounds: ", votes['round_name'].nunique())
print("Number of txs: ", votes['voter'].nunique())
print("Number of unique rows: ", votes.shape[0])
print("Number of unique project names: ", votes['project_name'].nunique())
print("Sum of amountUSD: ", votes['amountUSD'].sum())


Number of unique rounds:  18
Number of txs:  44467
Number of unique rows:  315775
Number of unique project names:  735
Sum of amountUSD:  602628.42804561


In [34]:
grouped_votes = votes.groupby('round_name').agg(
    unique_voters=pd.NamedAgg(column='voter', aggfunc='nunique'),
    sum_amount_usd=pd.NamedAgg(column='amountUSD', aggfunc='sum'),
    total_rows=pd.NamedAgg(column='voter', aggfunc='count'),
    unique_projects=pd.NamedAgg(column='project_name', aggfunc='nunique')
)
grouped_votes

,unique_voters,sum_amount_usd,total_rows,unique_projects
round_name,,,,
1inch LatAm,902,2800.976316,1701,12
Arbitrum Citizens Retrofunding Round 1,2847,12129.897348,7106,25
Climate Solutions Round,3375,35313.970085,14559,100
"Ethereum Core Infrastructure, Research, and Development",14406,58730.371082,82364,28
Ethereum.org Contributors Round,294,1276.901714,579,19
Global Chinese Community Round at GG19,1008,5358.485014,2300,28
Governance Research Round,1061,7118.047696,2391,34
GreenPill Network Round,172,988.251502,403,12
OpenCivics Genesis Round,353,8703.364932,710,15


In [35]:
eth_model = pd.read_csv('data/sample_GG19_model_score.csv').drop(columns=['Unnamed: 0'])
eth_model

,address,eth_model_score
0,0x0000fe532e6ee3ee20d43a0d138537a1ab26be57,0.0
1,0x00030cb9f625573d78e1b59393e6f052e6706e58,0.0
2,0x000d839d5cb48327c5c9f8143c78b51c1a757bb9,15.0
3,0x000dbf2733da51135c1b21c8ef71a3d474383f0d,0.0
4,0x000e2dbe4ca6ae1281e12f6599a06dc949c5219d,3.0
...,...,...
10532,0xffe6d9d29480d0673c4fc46da22d21e245c8e731,1.0
10533,0xfff04f139cd90cc696bf66f6af40c87be163b5fc,0.0
10534,0xfff81df58b94d3d4a4e07a77d6bf2f5c3034b092,85.0
10535,0xfffcfe7a5d540e51ec8086af6d7c8a053beee592,0.0


In [36]:
# Plotting histogram with log scale y
fig = px.histogram(eth_model, x="eth_model_score", log_y=True,  nbins=101)
fig.update_layout(title_text='Voter Frequency by ETH Model Score', xaxis_title='Eth Model Score', yaxis_title='Count of Voters (log scale)')
fig.update_traces(marker_color='darkblue')

fig.show()


In [38]:
# Find out how many of the address in eth_model are in the voter column of votes
common_addresses = eth_model[eth_model['address'].isin(votes['voter'])]
print("Number of common addresses: ", common_addresses.shape[0])


Number of common addresses:  10537


In [39]:
filtered_votes = votes[votes['voter'].isin(eth_model['address'])]
print("Number of voters left in filtered_votes: ", filtered_votes['voter'].nunique())

Number of voters left in filtered_votes:  10537


In [40]:
# Grouping by score and counting unique voters
# Creating a new column 'floor_score' in filtered_votes by applying the floor function to 'score'
filtered_votes['floor_score'] = filtered_votes['score'].apply(np.floor)


grouped_filtered_votes_score = filtered_votes.groupby('floor_score').agg(
    unique_voters_filtered=pd.NamedAgg(column='voter', aggfunc='nunique')
)

# Plotting histogram with log scale y
fig = px.histogram(grouped_filtered_votes_score, x=grouped_filtered_votes_score.index, y="unique_voters_filtered", log_y=False, nbins=69)
fig.update_layout(title_text='Voter Frequency by Passport Score', xaxis_title='Passport Score', yaxis_title='Count of Voters')
fig.update_traces(marker_color='darkblue')

fig.show()




/var/folders/vc/4gnlrkhx1bqgc_p_rhn78pxh0000gn/T/ipykernel_34492/3591272125.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [42]:
# This code groups filtered_votes by round_name and counts unique voters, merges grouped_votes and grouped_filtered_votes on round_name,
# fills NaN values with 0, calculates the percentage of overall unique voters in each round_name which remain in filtered_votes,
# selects the required columns, and prints the output.

grouped_filtered_votes = filtered_votes.groupby('round_name').agg(
    unique_voters_filtered=pd.NamedAgg(column='voter', aggfunc='nunique')
)
merged_votes = pd.merge(grouped_votes, grouped_filtered_votes, on='round_name', how='left')
merged_votes['unique_voters_filtered'].fillna(0, inplace=True)
merged_votes['percent_of_voters_in_filter'] = (merged_votes['unique_voters_filtered'] / merged_votes['unique_voters']) * 100
output = merged_votes[['unique_voters', 'unique_voters_filtered', 'percent_of_voters_in_filter']]
output


,unique_voters,unique_voters_filtered,percent_of_voters_in_filter
round_name,,,
1inch LatAm,902,207,22.949002
Arbitrum Citizens Retrofunding Round 1,2847,710,24.938532
Climate Solutions Round,3375,766,22.696296
"Ethereum Core Infrastructure, Research, and Development",14406,3414,23.698459
Ethereum.org Contributors Round,294,74,25.170068
Global Chinese Community Round at GG19,1008,238,23.611111
Governance Research Round,1061,259,24.410933
GreenPill Network Round,172,36,20.930233
OpenCivics Genesis Round,353,92,26.062323


In [44]:
passport_score_threshold = 15
eth_model_score_threshold = 50

merged_filtered_votes = pd.merge(filtered_votes, eth_model, left_on='voter', right_on='address', how='left')
filtered_voters_and_scores = merged_filtered_votes.groupby('voter').agg(
    score=pd.NamedAgg(column='score', aggfunc='first'),
    eth_model_score=pd.NamedAgg(column='eth_model_score', aggfunc='mean')
)
filtered_voters_and_scores

top_right = filtered_voters_and_scores[(filtered_voters_and_scores['score'] >= passport_score_threshold) & (filtered_voters_and_scores['eth_model_score'] > eth_model_score_threshold)]
top_left = filtered_voters_and_scores[(filtered_voters_and_scores['score'] < passport_score_threshold) & (filtered_voters_and_scores['eth_model_score'] >= eth_model_score_threshold)]
bottom_left = filtered_voters_and_scores[(filtered_voters_and_scores['score'] < passport_score_threshold) & (filtered_voters_and_scores['eth_model_score'] < eth_model_score_threshold)]
bottom_right = filtered_voters_and_scores[(filtered_voters_and_scores['score'] >= passport_score_threshold) & (filtered_voters_and_scores['eth_model_score'] <= eth_model_score_threshold)]
total_records = len(filtered_voters_and_scores)

fig = px.scatter(filtered_voters_and_scores, x='score', y='eth_model_score', title='Passport Score vs Eth Model Score')
fig.update_traces(marker=dict(
    color=filtered_voters_and_scores['eth_model_score'],
    colorscale='Viridis',  # Changed colorscale to 'Viridis' which is more colorblind friendly
    colorbar=dict(title='Eth Model Score'),
    cmin=0,
    cmax=100
))
fig.update_layout(xaxis_title='Passport Score', yaxis_title='Eth Model Score')

# Adding lines where x = 15 and y = 50
fig.add_shape(
    type="line", line=dict(dash='dash'),
    x0=passport_score_threshold, x1=passport_score_threshold, y0=0, y1=100
)
fig.add_shape(
    type="line", line=dict(dash='dash'),
    x0=0, x1=70, y0=eth_model_score_threshold, y1=eth_model_score_threshold
)

# Calculate the percentage of records in each quadrant
top_right_percent = (len(top_right)/total_records)*100
top_left_percent = (len(top_left)/total_records)*100
bottom_left_percent = (len(bottom_left)/total_records)*100
bottom_right_percent = (len(bottom_right)/total_records)*100

# Add annotations to each quadrant
fig.add_annotation(x=30, y=75, text=f"{top_right_percent:.2f}%", showarrow=False, 
                   font=dict(color="black", size=14), 
                   bgcolor="rgba(255, 255, 255, 0.6)")
fig.add_annotation(x=5, y=75, text=f"{top_left_percent:.2f}%", showarrow=False, 
                   font=dict(color="black", size=14), 
                   bgcolor="rgba(255, 255, 255, 0.6)")
fig.add_annotation(x=5, y=eth_model_score_threshold-10, text=f"{bottom_left_percent:.2f}%", showarrow=False, 
                   font=dict(color="black", size=14), 
                   bgcolor="rgba(255, 255, 255, 0.6)")
fig.add_annotation(x=30, y=eth_model_score_threshold-10, text=f"{bottom_right_percent:.2f}%", showarrow=False, 
                   font=dict(color="black", size=14), 
                   bgcolor="rgba(255, 255, 255, 0.6)")
fig.show()



In [45]:
correlation = filtered_voters_and_scores['score'].corr(filtered_voters_and_scores['eth_model_score'])
print(f"Correlation between score and eth_model_score is: {correlation}")


Correlation between score and eth_model_score is: 0.16940807636733837


In [46]:
def calculate_score_thresholds(df, thresholds, variable_name='eth_model_score'):
    results = []
    total_addresses = df.shape[0]
    for threshold in thresholds:
        less_than_threshold = df[df[variable_name] < threshold]
        num_less_than_threshold = less_than_threshold.shape[0]
        percent_less_than_threshold = (num_less_than_threshold / total_addresses) * 100
        results.append([threshold, num_less_than_threshold, percent_less_than_threshold])
    return pd.DataFrame(results, columns=['Threshold', 'Number of Addresses', 'Percent of Addresses'])

# Define the thresholds
thresholds = [1,2, 25, 50, 75, 90]

# Calculate the score thresholds and display the results
score_thresholds_df = calculate_score_thresholds(eth_model, thresholds)
score_thresholds_df



,Threshold,Number of Addresses,Percent of Addresses
0,1,5672,53.829363
1,2,6967,66.119389
2,25,8950,84.938787
3,50,9350,88.734934
4,75,9666,91.733890
5,90,9906,94.011578


In [47]:
thresholds = [0, 15,20, 25]
score_thresholds_df = calculate_score_thresholds(filtered_voters_and_scores, thresholds, variable_name='score')


In [91]:
rounds = pd.read_csv('data/rounds_summary.csv')
rounds

,Unnamed: 0,round_name,amountUSD,votes,uniqueContributors,chain_id,round_id,has_matching_cap,matching_cap_amount,matching_funds_available,token,has_min_donation_threshold,min_donation_threshold_amount,sybilDefense
0,0,Vault Round 1,470425,1282,792,250,0x8dce7a66e0c310f9f89e847dba83b2344d589161,True,15.00,750000.0000,0x0000000000000000000000000000000000000000,True,1.000,True
1,1,Web3 Open Source Software,362713,202303,39707,10,0x8de918f0163b2021839a8d84954dd7e8e151326d,True,5.00,300000.0000,0xda10009cbd5d07dd0cecc66161fc93d7c9000da1,True,0.960,True
2,2,Web3 Open Source Software Round,330690,81811,14554,1,0x12bb5bbbfe596dbc489d209299b8302c3300fa40,True,4.00,350000.0000,0x6b175474e89094c44da98b954eedeac495271d0f,True,1.000,True
3,3,Web3 Open Source Software,297068,133861,26250,424,0xd4cc0dd193c7dc1d665ae244ce12d7fab337a008,True,7.42,200000.0000,0x6c121674ba6736644a7e73a8741407fe8a5ee5ba,True,0.969,True
4,4,Web3 Community and Education,138609,52293,16707,424,0x98720dd1925d34a2453ebc1f91c9d48e7e89ec29,True,7.42,200000.0000,0x6c121674ba6736644a7e73a8741407fe8a5ee5ba,True,0.969,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,106,"NEAR 2.0, META POOL & GITCOIN",4,2,2,10,0x79115c9114055f16bb5b0e9bbfa450844d0fcb3a,True,10.00,30000.0000,0xda10009cbd5d07dd0cecc66161fc93d7c9000da1,True,0.950,True
107,107,Hokkien Mee Improvements,2,2,1,1,0x24ffd26b727949e3523a8e62d145057cf13a44d0,False,50.00,5.0000,0x6b175474e89094c44da98b954eedeac495271d0f,False,1.000,False
108,108,Public Good Meme Round 1,2,3,1,10,0x55e3ff2966fbad6a4322d2b9223be77dca9b89d3,True,10.00,100.0000,0xda10009cbd5d07dd0cecc66161fc93d7c9000da1,False,NaN,False
109,109,Scroll guide,1,2,2,137,0xfd69e398d0e1b4ff78f22b6f615d1cf66a9e18e4,True,30.00,0.5000,0x0000000000000000000000000000000000000000,False,0.000,True


In [92]:
votes.head(1)

,voter,project_name,transaction,blockNumber,projectId,payoutAddress,token,amount,amountUSD,amountRoundToken,raw_timestamp,tx_timestamp,round_name,chain_id,roundAddress,min_donation_threshold_amount,score,stamps,scoreTimestamp,updatedAt
0,0x5447cb0a93456e1f7da48cd4a76dab8a9657e4f5,ECWireless | RaidGuild,0x36bbe8caa9054218d9a2c781ec85ee9350045973c1c0...,112036180,2232,0x1a9cEe6E1D21c3C09FB83A980EA54299f01920cd,0x0000000000000000000000000000000000000000,2000000000000000,4.129054,4130723720000000000,1699671137,2023-11-11T02:52:17,Raid Guild Contributors v0,10,0xB5C0939A9BB0C404b028D402493b86D9998af55e,0.96,18.9175,"{""Ens"": 2.2, ""NFT"": 0.69, ""Google"": 1.01, ""ZkS...",2023-11-11T14:26:19,2023-12-06T12:57:08


In [93]:
ROUND_SELECTION = 'Web3 Community and Education'
votes_round = votes[votes['round_name'] == ROUND_SELECTION].copy()
# Filter rounds using round_id from votes_round roundAddress
if not votes_round.empty:
    round_id = votes_round['roundAddress'].str.lower().unique()[0]
    print(round_id)
    filtered_rounds = rounds[rounds['round_id'].str.lower() == round_id]
    filtered_rounds

    matching_cap_amount = filtered_rounds['matching_cap_amount'].astype(float).values[0] if 'matching_cap_amount' in filtered_rounds and not pd.isnull(filtered_rounds['matching_cap_amount'].values[0]) else 'No Cap'
    matching_amount = filtered_rounds['matching_funds_available'].astype(float).values[0] if 'matching_funds_available' in filtered_rounds else 0
    min_donation_threshold_amount = filtered_rounds['min_donation_threshold_amount'].astype(float).values[0] if 'min_donation_threshold_amount' in filtered_rounds and not pd.isnull(filtered_rounds['min_donation_threshold_amount'].values[0]) else 0.0

    print(f"Round Name: {ROUND_SELECTION}")
    print(f"Matching Cap Amount: {matching_cap_amount}%")
    print(f"Matching Amount: {matching_amount} ")
    print(f"Minimum Donation Threshold Amount: ${min_donation_threshold_amount}")
    print("Number of voters: ", votes_round['voter'].nunique())
    print("Number of unique rows: ", votes_round.shape[0])
    print("Number of unique project names: ", votes_round['project_name'].nunique())
    print("Sum of amountUSD: ", votes_round['amountUSD'].sum())
else:
    print("No votes found for the selected round.")


0x98720dd1925d34a2453ebc1f91c9d48e7e89ec29
Round Name: Web3 Community and Education
Matching Cap Amount: 7.42%
Matching Amount: 200000.0 
Minimum Donation Threshold Amount: $0.969
Number of voters:  16707
Number of unique rows:  52284
Number of unique project names:  310
Sum of amountUSD:  138673.63636884


In [55]:
# Filter votes_round to voters in eth_model['addresses']
filtered_votes_round = votes_round[votes_round['voter'].isin(eth_model['address'])].copy()

# Join eth_model_score to filtered_votes_round
eth_model_votes = pd.merge(filtered_votes_round, eth_model, left_on='voter', right_on='address', how='left')
voter_score_summary = eth_model_votes.groupby('voter').agg(
    score=pd.NamedAgg(column='score', aggfunc='first'),
    eth_model_score=pd.NamedAgg(column='eth_model_score', aggfunc='first'),
    amountUSD=pd.NamedAgg(column='amountUSD', aggfunc='sum')
)
# Drop the 'score' column
eth_model_votes.drop('score', axis=1, inplace=True)
# Rename 'eth_model_score' to 'score'
eth_model_votes.rename(columns={'eth_model_score': 'score'}, inplace=True)


In [56]:
voter_score_summary.shape

(4031, 3)

In [57]:
no_passport_filtered_votes_round = filtered_votes_round.copy()
no_passport_filtered_votes_round['score'] = 25

In [94]:
strategies = ['qf', 'COCM']
datasets = [votes_round]
dataset_names = ['votes_round']

# Initialize matching_df
matching_df = pd.DataFrame()

# Iterate over datasets and strategies
for i, dataset in enumerate(datasets):
    dataset_prep = fundingutils.prep_donations_data(dataset, min_donation_threshold_amount, 15)
    dataset_matrix = fundingutils.pivot_votes(dataset_prep)
    for strategy in strategies:
        votes_qf_matching = fundingutils.get_qf_matching(strategy, dataset_matrix, matching_cap_amount, matching_amount, cluster_df = None if strategy == 'qf' else dataset_matrix)
        votes_qf_matching = votes_qf_matching.rename(columns={'project_name': 'Project', 'matching_amount': 'Matching Amount'})
        votes_qf_matching['Strategy'] = strategy.capitalize() if strategy != 'COCM' else 'COCM'
        votes_qf_matching['Dataset'] = dataset_names[i]
        matching_df = pd.concat([matching_df, votes_qf_matching])

matching_df.sort_values(by='Matching Amount', ascending=False, inplace=True)
matching_df
# OPTIONAL: save the dataframe to a csv file
#matching_df.to_csv(f'data/gg19_{ROUND_SELECTION}_matching_df.csv', index=False)


/Users/umarkhan/qf_insights/fundingutils.py:32: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



,Project,Matching Amount,matching_percent,Strategy,Dataset
296,ZachXBT,14840.000000,7.420000,Qf,votes_round
261,Trustalabs,14840.000000,7.420000,COCM,votes_round
293,Wizard Bridge EVM,14840.000000,7.420000,Qf,votes_round
113,EtherScore,14840.000000,7.420000,Qf,votes_round
157,Hypercerts Foundation,14840.000000,7.420000,Qf,votes_round
...,...,...,...,...,...
109,Empowering Web3 Education with GetCrypto Card ...,1.658981,0.000829,Qf,votes_round
214,Poly Raiders (Proof of Charity),1.351947,0.000676,Qf,votes_round
150,Greenpill Kenya,1.338522,0.000669,Qf,votes_round
170,Learn Self-Custody with irish,1.010734,0.000505,Qf,votes_round


In [58]:
strategies = ['qf', 'COCM']
datasets = [votes_round, filtered_votes_round, eth_model_votes, no_passport_filtered_votes_round]
dataset_names = ['votes_round', 'filtered_votes_round', 'eth_model_votes', 'no_passport_filtered_votes_round']

# Initialize matching_df
matching_df = pd.DataFrame()

# Iterate over datasets and strategies
for i, dataset in enumerate(datasets):
    dataset_prep = fundingutils.prep_donations_data(dataset, min_donation_threshold_amount, 15)
    dataset_matrix = fundingutils.pivot_votes(dataset_prep)
    for strategy in strategies:
        votes_qf_matching = fundingutils.get_qf_matching(strategy, dataset_matrix, matching_cap_amount, matching_amount, cluster_df = None if strategy == 'qf' else dataset_matrix)
        votes_qf_matching = votes_qf_matching.rename(columns={'project_name': 'Project', 'matching_amount': 'Matching Amount'})
        votes_qf_matching['Strategy'] = strategy.capitalize() if strategy != 'COCM' else 'COCM'
        votes_qf_matching['Dataset'] = dataset_names[i]
        matching_df = pd.concat([matching_df, votes_qf_matching])

matching_df.sort_values(by='Matching Amount', ascending=False, inplace=True)
matching_df
# OPTIONAL: save the dataframe to a csv file
matching_df.to_csv(f'data/gg19_{ROUND_SELECTION}_matching_df.csv', index=False)


/Users/umarkhan/qf_insights/fundingutils.py:32: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

/Users/umarkhan/qf_insights/fundingutils.py:32: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

/Users/umarkhan/qf_insights/fundingutils.py:32: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

/Users/umarkhan/qf_insights/fundingutils.py:32: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



In [126]:
matching_df = pd.read_csv('data/gg19_Web3 Community and Education_matching_df.csv')
matching_df['Strategy'] = matching_df['Strategy'] + "_" + matching_df['Dataset']
matching_df.head(2)


,Project,Matching Amount,matching_percent,Strategy,Dataset
0,Trustalabs,14840.0,7.42,COCM_no_passport_filtered_votes_round,no_passport_filtered_votes_round
1,Metopia,14840.0,7.42,Qf_filtered_votes_round,filtered_votes_round


In [6]:
# Pivot the matching_df to get the matching amounts for each project per strategy-dataset combination
pivot_matching_df = matching_df.pivot_table(index='Project', columns=['Strategy'], values='Matching Amount')

# Reset the index
pivot_matching_df.reset_index(inplace=True)


# Display the pivot table
pivot_matching_df.head(20)



Strategy,Project,COCM_eth_model_votes,COCM_filtered_votes_round,COCM_no_passport_filtered_votes_round,COCM_votes_round,Qf_eth_model_votes,Qf_filtered_votes_round,Qf_no_passport_filtered_votes_round,Qf_votes_round
0,DAOESCO & DAO Palace,709.797988,592.473692,717.652958,279.829475,93.577133,60.668803,87.120901,24.468575
1,1c4m3by ScamHunt,163.621449,125.821249,175.002430,552.667127,43.351193,13.838864,22.019429,130.202888
2,4Seas DeSoc,628.515876,446.007784,459.983902,310.500100,126.257124,62.777064,68.751643,33.288189
3,A Nacion Bankless Perspective,605.081052,571.946637,534.413978,624.388738,118.594069,88.788216,132.211332,100.850987
4,AA Chinese Community,343.519689,325.326856,454.725700,254.867876,37.526130,29.290123,44.541436,23.096742
5,AGRii DAO,107.863429,141.141984,162.672567,215.913600,2.003943,9.957834,17.851245,17.210247
6,AQKU Blockchain para Peru,51.407743,139.497616,159.257732,204.634262,0.088062,10.295199,13.769693,12.624111
7,AW House,12.409647,94.620484,117.736227,183.605976,5.560845,15.849085,16.937689,17.426076
8,Admiano,48.380052,128.624333,98.222785,170.142132,0.000000,12.382816,9.680144,13.427039
9,Afro Blockchain Carnival,0.000000,8.554283,6.962763,76.222676,0.000000,0.741250,0.619013,3.543114


In [127]:
legit_scores = pd.read_csv('data/gg19community_legitscores.csv')
legit_scores.head

<bound method NDFrame.head of                                                Project  legitimacy_score
0                                    Wizard Bridge EVM                 1
1                                Hypercerts Foundation                 5
2                                    Olimpio Education                 2
3                                              Metopia                 2
4                                              ZachXBT                 5
..                                                 ...               ...
305  Empowering Web3 Education with GetCrypto Card ...                 3
306                    Poly Raiders (Proof of Charity)                 3
307                                    Greenpill Kenya                 3
308                      Learn Self-Custody with irish                 3
309                                   inDemniFi Crypto                 3

[310 rows x 2 columns]>

In [63]:
#legit_scores = pd.read_csv('data/gg19_community_likelihood.csv')
# Rename 'human_likelihood' column to 'legitimacy_score'
#legit_scores.rename(columns={'human_likelihood': 'legitimacy_score'}, inplace=True)

#legit_scores.head(10)

,Project,legitimacy_score
0,Wizard Bridge EVM,1
1,Hypercerts Foundation,5
2,Olimpio Education,2
3,Metopia,2
4,ZachXBT,5
5,Trustalabs,2
6,EtherScore,2
7,Alpha Insiders,1
8,EtherDrops Bot,2
9,Onlyfun,1


In [114]:
legit_scores.shape[0]

310

In [128]:
matching_df['simple_project_name'] = matching_df['Project'].apply(lambda x: ''.join(e for e in x if e.isalnum() and ord(e) < 128).lower())
legit_scores['simple_project_name'] = legit_scores['Project'].apply(lambda x: ''.join(e for e in x if e.isalnum() and ord(e) < 128).lower())
legit_scores.drop(columns=['Project'], inplace=True)
matching_df_joined = pd.merge(matching_df, legit_scores, how='left', left_on='simple_project_name', right_on='simple_project_name')

matching_df_joined = pd.merge(matching_df, legit_scores, on='simple_project_name', how='left')
matching_df_joined.head(20)

,Project,Matching Amount,matching_percent,Strategy,Dataset,simple_project_name,legitimacy_score
0,Trustalabs,14840.0,7.42,COCM_no_passport_filtered_votes_round,no_passport_filtered_votes_round,trustalabs,2
1,Metopia,14840.0,7.42,Qf_filtered_votes_round,filtered_votes_round,metopia,2
2,Olimpio Education,14840.0,7.42,Qf_votes_round,votes_round,olimpioeducation,2
3,Trustalabs,14840.0,7.42,Qf_no_passport_filtered_votes_round,no_passport_filtered_votes_round,trustalabs,2
4,Wizard Bridge EVM,14840.0,7.42,Qf_filtered_votes_round,filtered_votes_round,wizardbridgeevm,1
5,Trustalabs,14840.0,7.42,Qf_eth_model_votes,eth_model_votes,trustalabs,2
6,Olimpio Education,14840.0,7.42,Qf_no_passport_filtered_votes_round,no_passport_filtered_votes_round,olimpioeducation,2
7,Hypercerts Foundation,14840.0,7.42,Qf_filtered_votes_round,filtered_votes_round,hypercertsfoundation,5
8,Metopia,14840.0,7.42,Qf_no_passport_filtered_votes_round,no_passport_filtered_votes_round,metopia,2
9,Hypercerts Foundation,14840.0,7.42,Qf_no_passport_filtered_votes_round,no_passport_filtered_votes_round,hypercertsfoundation,5


In [108]:
matching_df_joined.isnull().sum()

Project                0
Matching Amount        0
matching_percent       0
Strategy               0
Dataset                0
simple_project_name    0
legitimacy_score       0
dtype: int64

In [129]:
# SET GRAPH FORMATTING

strategies = ['Qf_votes_round', 'Qf_filtered_votes_round', 'Qf_eth_model_votes', 'Qf_no_passport_filtered_votes_round',
              'COCM_votes_round', 'COCM_filtered_votes_round', 'COCM_eth_model_votes', 'COCM_no_passport_filtered_votes_round']
names = ['QF - All', 'QF - Passport', 'QF - Eth Model', 'QF - No Passport',  # Names for the legend
         'COCM - All', 'COCM - Passport', 'COCM - Eth Model', 'COCM - No Passport']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#bcbd22', '#7f7f7f', '#17becf', '#9467bd']  # Colors for the lines

font = dict(
            family="Arial, sans-serif",
            size=14,
            color="DarkSlateGrey"
        )
xaxis = dict(
            title_standoff=20,
            automargin=True,
            tickfont=dict(size=10),
            tickangle=45
        )
yaxis = dict(
            title_standoff=20,
            automargin=True 
        )


In [125]:
grouped_df = matching_df_joined.groupby(['legitimacy_score', 'Strategy'])['Matching Amount'].sum().reset_index()
grouped_df

,legitimacy_score,Strategy,Matching Amount
0,1,COCM_eth_model_votes,5954.329585
1,1,COCM_filtered_votes_round,8310.783530
2,1,COCM_no_passport_filtered_votes_round,7613.208866
3,1,COCM_votes_round,7740.640024
4,1,Qf_eth_model_votes,22245.852661
5,1,Qf_filtered_votes_round,34337.828097
6,1,Qf_no_passport_filtered_votes_round,33759.985368
7,1,Qf_votes_round,35381.920840
8,2,COCM_eth_model_votes,21897.570809
9,2,COCM_filtered_votes_round,37623.149748


In [118]:
# Calculate the total Matching Amount for each Strategy
total_matching_amount = grouped_df.groupby('Strategy')['Matching Amount'].sum()
print(total_matching_amount)

# Calculate the percent of Matching Amount per strategy assigned to each legitimacy_score
grouped_df['Percent of Matching Amount'] = grouped_df.apply(lambda row: (row['Matching Amount'] / total_matching_amount[row['Strategy']]) * 100, axis=1)
grouped_df


Strategy
COCM_eth_model_votes                     200000.0
COCM_filtered_votes_round                200000.0
COCM_no_passport_filtered_votes_round    200000.0
COCM_votes_round                         200000.0
Qf_eth_model_votes                       200000.0
Qf_filtered_votes_round                  200000.0
Qf_no_passport_filtered_votes_round      200000.0
Qf_votes_round                           200000.0
Name: Matching Amount, dtype: float64


,legitimacy_score,Strategy,Matching Amount,Percent of Matching Amount
0,1,COCM_eth_model_votes,5954.329585,2.977165
1,1,COCM_filtered_votes_round,8310.783530,4.155392
2,1,COCM_no_passport_filtered_votes_round,7613.208866,3.806604
3,1,COCM_votes_round,7740.640024,3.870320
4,1,Qf_eth_model_votes,22245.852661,11.122926
5,1,Qf_filtered_votes_round,34337.828097,17.168914
6,1,Qf_no_passport_filtered_votes_round,33759.985368,16.879993
7,1,Qf_votes_round,35381.920840,17.690960
8,2,COCM_eth_model_votes,21897.570809,10.948785
9,2,COCM_filtered_votes_round,37623.149748,18.811575


In [103]:
grouped_df

,legitimacy_score,Strategy,Matching Amount


In [119]:
strategies_to_keep = ['Qf_votes_round','COCM_votes_round']
grouped_df = grouped_df[grouped_df['Strategy'].isin(strategies_to_keep)]
names = ['QF - All', 'COCM - All']
colors = ['#1f77b4', '#ff7f0e']

# Pivot the data to have strategies as columns and legitimacy_scores as rows
pivot_df = grouped_df.pivot(index='legitimacy_score', columns='Strategy', values='Percent of Matching Amount')

fig = go.Figure()
for strategy, color, name in zip(pivot_df.columns, colors, names):
    fig.add_trace(go.Bar(x=pivot_df.index, y=pivot_df[strategy], name=name, marker_color=color))

fig.update_layout(
    title='Percent of Matching Amount per Strategy for each Legitimacy Score',
    xaxis_title='Legitimacy Score',
    yaxis_title='Percent of Matching Amount'
)

fig.show()

In [244]:

matching_df.head(10)

,Project,Matching Amount,matching_percent,Strategy,Dataset
0,Trustalabs,14840.0,7.42,COCM_no_passport_filtered_votes_round,no_passport_filtered_votes_round
1,Metopia,14840.0,7.42,Qf_filtered_votes_round,filtered_votes_round
2,Olimpio Education,14840.0,7.42,Qf_votes_round,votes_round
3,Trustalabs,14840.0,7.42,Qf_no_passport_filtered_votes_round,no_passport_filtered_votes_round
4,Wizard Bridge EVM,14840.0,7.42,Qf_filtered_votes_round,filtered_votes_round
5,Trustalabs,14840.0,7.42,Qf_eth_model_votes,eth_model_votes
6,Olimpio Education,14840.0,7.42,Qf_no_passport_filtered_votes_round,no_passport_filtered_votes_round
7,Hypercerts Foundation,14840.0,7.42,Qf_filtered_votes_round,filtered_votes_round
8,Metopia,14840.0,7.42,Qf_no_passport_filtered_votes_round,no_passport_filtered_votes_round
9,Hypercerts Foundation,14840.0,7.42,Qf_no_passport_filtered_votes_round,no_passport_filtered_votes_round


In [130]:
# Get the top 15 projects with the highest Matching Amount in the Qf_filtered_votes_round Strategy
top_15_projects = matching_df[matching_df['Strategy'] == 'Qf_filtered_votes_round'].nlargest(15, 'Matching Amount')['Project'].values

# Filter the matching_df to only these projects (but for all strategies)
matching_df = matching_df[matching_df['Project'].isin(top_15_projects)]


In [131]:
matching_df

,Project,Matching Amount,matching_percent,Strategy,Dataset,simple_project_name
0,Trustalabs,14840.000000,7.420000,COCM_no_passport_filtered_votes_round,no_passport_filtered_votes_round,trustalabs
1,Metopia,14840.000000,7.420000,Qf_filtered_votes_round,filtered_votes_round,metopia
2,Olimpio Education,14840.000000,7.420000,Qf_votes_round,votes_round,olimpioeducation
3,Trustalabs,14840.000000,7.420000,Qf_no_passport_filtered_votes_round,no_passport_filtered_votes_round,trustalabs
4,Wizard Bridge EVM,14840.000000,7.420000,Qf_filtered_votes_round,filtered_votes_round,wizardbridgeevm
...,...,...,...,...,...,...
373,Onlyfun,748.651687,0.374326,COCM_no_passport_filtered_votes_round,no_passport_filtered_votes_round,onlyfun
405,Onlyfun,693.134648,0.346567,COCM_filtered_votes_round,filtered_votes_round,onlyfun
411,Onlyfun,683.386422,0.341693,Qf_eth_model_votes,eth_model_votes,onlyfun
538,Biteye,531.540629,0.265770,COCM_eth_model_votes,eth_model_votes,biteye


In [132]:
# Define a mapping of strategy names to display names
strategy_display_names = {
    'Qf_filtered_votes_round': 'QF - Passport',
    'COCM_filtered_votes_round': 'COCM - Passport',
    'Qf_eth_model_votes': 'QF - Eth Model',
    'COCM_eth_model_votes': 'COCM - Eth Model',
    'Qf_no_passport_filtered_votes_round': 'QF - No Passport',
    'COCM_no_passport_filtered_votes_round': 'COCM - No Passport',
}

def generate_percent_difference_bargraph(df, strategies):
    # Filter df to only include rows with the given strategies
    df = df[df['Strategy'].isin(strategies)]
    df = df.sort_values(by='Matching Amount')
    df['Project_Short'] = df['Project'].apply(lambda x: x[:15] + '...' if len(x) > 15 else x)
    
    # Pivot the data to have strategies as columns and projects as rows
    df_pivot = df.pivot(index='Project_Short', columns='Strategy', values='Matching Amount')
    
    # Calculate percentage differences
    base_strategy = 'Qf_filtered_votes_round'
    for strategy in strategies:
        if strategy != base_strategy:
            df_pivot[f'Percent_Diff_{strategy}'] = ((df_pivot[strategy] - df_pivot[base_strategy]) / 
                                                     df_pivot[base_strategy]) * 100
    
    # Melt the data back to long format
    df_melt = df_pivot.reset_index().melt(id_vars=['Project_Short'], 
                                          value_vars=[col for col in df_pivot.columns if 'Percent_Diff_' in col],
                                          var_name='Strategy', value_name='Percent_Diff')
    df_melt['Strategy'] = df_melt['Strategy'].apply(lambda x: x.replace('Percent_Diff_', ''))
    
    # Create a figure
    fig = go.Figure()
    
    # Add a bar plot for each strategy (excluding the base strategy) with custom colors
    for strategy in strategies:
        if strategy != base_strategy:
            strategy_df = df_melt[df_melt['Strategy'] == strategy]
            trace = go.Bar(x=strategy_df['Project_Short'], y=strategy_df['Percent_Diff'],
                           name=strategy_display_names[strategy], hovertext=strategy_df['Project_Short'],
                           marker=dict(color=colors[strategies.index(strategy)]))
            fig.add_trace(trace)
    
    fig.update_layout(
        title={
            'text': f'Web3 Community and Education QF Matching Comparison',
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(size=18, color='DarkSlateGrey')
        },
        xaxis_title='Project',
        yaxis_title='Percent Difference from QF - Passport',
        #height=600,
        font=font,
        xaxis=xaxis,
        yaxis=dict(ticksuffix='%'),
        legend_title='Matching Strategies',
        legend=dict(
            yanchor="top",
            y=1.0,
            xanchor="right",
            x=1.33,
            bgcolor='rgba(255, 255, 255, 0.5)'
        )
    )
    
    return df_melt, fig

# Define your groups as lists of strategies
strategies_to_compare = ['Qf_filtered_votes_round', 'COCM_filtered_votes_round', 'Qf_eth_model_votes']

# Use the function to filter data and plot graph for each group
result_df, percent_difference_fig = generate_percent_difference_bargraph(matching_df, strategies_to_compare)
percent_difference_fig.show()
result_df.head(10)

,Project_Short,Strategy,Percent_Diff
0,Alpha Insiders,COCM_filtered_votes_round,-75.219398
1,Biteye,COCM_filtered_votes_round,-75.914723
2,Castle Capital,COCM_filtered_votes_round,-53.347263
3,EtherDrops Bot,COCM_filtered_votes_round,-84.557831
4,EtherScore,COCM_filtered_votes_round,-5.612337
5,Giveth,COCM_filtered_votes_round,23.069562
6,Hypercerts Foun...,COCM_filtered_votes_round,-70.056414
7,Metopia,COCM_filtered_votes_round,-87.651497
8,Olimpio Educati...,COCM_filtered_votes_round,-78.324433
9,Onlyfun,COCM_filtered_votes_round,-83.422634


In [135]:
matching_df_joined

,Project,Matching Amount,matching_percent,Strategy,Dataset,simple_project_name,legitimacy_score
0,Trustalabs,1.484000e+04,7.420000e+00,COCM_no_passport_filtered_votes_round,no_passport_filtered_votes_round,trustalabs,2
1,Metopia,1.484000e+04,7.420000e+00,Qf_filtered_votes_round,filtered_votes_round,metopia,2
2,Olimpio Education,1.484000e+04,7.420000e+00,Qf_votes_round,votes_round,olimpioeducation,2
3,Trustalabs,1.484000e+04,7.420000e+00,Qf_no_passport_filtered_votes_round,no_passport_filtered_votes_round,trustalabs,2
4,Wizard Bridge EVM,1.484000e+04,7.420000e+00,Qf_filtered_votes_round,filtered_votes_round,wizardbridgeevm,1
...,...,...,...,...,...,...,...
2475,Learn Self-Custody with irish,0.000000e+00,0.000000e+00,Qf_eth_model_votes,eth_model_votes,learnselfcustodywithirish,3
2476,MKT Community,0.000000e+00,0.000000e+00,Qf_eth_model_votes,eth_model_votes,mktcommunity,3
2477,Nebula Web3 Brazil,0.000000e+00,0.000000e+00,Qf_eth_model_votes,eth_model_votes,nebulaweb3brazil,3
2478,Inbest Program,0.000000e+00,0.000000e+00,Qf_eth_model_votes,eth_model_votes,inbestprogram,3


In [141]:
# Define a mapping of strategy names to display names
strategy_display_names = {
    'Qf_filtered_votes_round': 'QF - Passport',
    'COCM_filtered_votes_round': 'COCM - Stamps',
    'Qf_eth_model_votes': 'QF - Eth Model',
    'COCM_eth_model_votes': 'COCM - Model',
    'Qf_no_passport_filtered_votes_round': 'QF - No Passport',
    'COCM_no_passport_filtered_votes_round': 'COCM - No Passport',
}

def generate_percent_difference_bargraph(df, strategies):
    # Filter df to only include rows with the given strategies
    df = df[df['Strategy'].isin(strategies)]
    df = df.sort_values(by='Matching Amount')
 
    grouped_df = df.groupby(['legitimacy_score', 'Strategy'])['Matching Amount'].sum().reset_index()
    # Pivot the data to have strategies as columns and projects as rows
    df_pivot = grouped_df.pivot(index='legitimacy_score', columns='Strategy', values='Matching Amount')
    
    # Calculate percentage differences
    base_strategy = 'Qf_filtered_votes_round'
    for strategy in strategies:
        if strategy != base_strategy:
            df_pivot[f'Percent_Diff_{strategy}'] = ((df_pivot[strategy] - df_pivot[base_strategy]) / 
                                                     df_pivot[base_strategy]) * 100
    
    # Melt the data back to long format
    df_melt = df_pivot.reset_index().melt(id_vars=['legitimacy_score'], 
                                          value_vars=[col for col in df_pivot.columns if 'Percent_Diff_' in col],
                                          var_name='Strategy', value_name='Percent_Diff')
    df_melt['Strategy'] = df_melt['Strategy'].apply(lambda x: x.replace('Percent_Diff_', ''))
    
    # Create a figure
    fig = go.Figure()
    
    # Add a bar plot for each strategy (excluding the base strategy) with custom colors
    for strategy in strategies:
        if strategy != base_strategy:
            strategy_df = df_melt[df_melt['Strategy'] == strategy]
            trace = go.Bar(x=strategy_df['legitimacy_score'], y=strategy_df['Percent_Diff'],
                           name=strategy_display_names[strategy], hovertext=strategy_df['legitimacy_score'],
                           marker=dict(color=colors[strategies.index(strategy)]))
            fig.add_trace(trace)
    
    fig.update_layout(
        title={
            'text': f'Web3 Community and Education QF Matching Comparison',
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(size=18, color='DarkSlateGrey')
        },
        xaxis_title='Legitimacy Score',
        yaxis_title='Percent Difference from QF - Passport',
        #height=600,
        font=font,
        xaxis=xaxis,
        yaxis=dict(ticksuffix='%'),
        legend_title='Matching Strategies',
        legend=dict(
            yanchor="top",
            y=1.0,
            xanchor="right",
            x=1.33,
            bgcolor='rgba(255, 255, 255, 0.5)'
        )
    )
    
    return df_melt, fig

# Define your groups as lists of strategies
strategies_to_compare = ['Qf_filtered_votes_round', 'COCM_filtered_votes_round', 'Qf_eth_model_votes']

# Use the function to filter data and plot graph for each group
result_df, percent_difference_fig = generate_percent_difference_bargraph(matching_df_joined, strategies_to_compare)
percent_difference_fig.show()
result_df.head(10)

,legitimacy_score,Strategy,Percent_Diff
0,1,COCM_filtered_votes_round,-75.797003
1,2,COCM_filtered_votes_round,-48.888175
2,3,COCM_filtered_votes_round,118.837084
3,4,COCM_filtered_votes_round,165.728633
4,5,COCM_filtered_votes_round,5.229655
5,1,Qf_eth_model_votes,-35.214736
6,2,Qf_eth_model_votes,-20.054766
7,3,Qf_eth_model_votes,-0.841099
8,4,Qf_eth_model_votes,52.948297
9,5,Qf_eth_model_votes,44.959587


In [142]:
# Define your groups as lists of strategies
strategies_to_compare = ['Qf_filtered_votes_round', 'COCM_filtered_votes_round', 'COCM_eth_model_votes']

# Use the function to filter data and plot graph for each group
result_df, percent_difference_fig = generate_percent_difference_bargraph(matching_df_joined, strategies_to_compare)
percent_difference_fig.show()
result_df.head(10)

,legitimacy_score,Strategy,Percent_Diff
0,1,COCM_filtered_votes_round,-75.797003
1,2,COCM_filtered_votes_round,-48.888175
2,3,COCM_filtered_votes_round,118.837084
3,4,COCM_filtered_votes_round,165.728633
4,5,COCM_filtered_votes_round,5.229655
5,1,COCM_eth_model_votes,-82.659563
6,2,COCM_eth_model_votes,-70.251698
7,3,COCM_eth_model_votes,112.924342
8,4,COCM_eth_model_votes,229.638199
9,5,COCM_eth_model_votes,32.260841


In [123]:
matching_df.head(10)

,Project,Matching Amount,matching_percent,Strategy,Dataset,simple_project_name
0,Trustalabs,14840.0,7.42,COCM_no_passport_filtered_votes_round,no_passport_filtered_votes_round,trustalabs
1,Metopia,14840.0,7.42,Qf_filtered_votes_round,filtered_votes_round,metopia
2,Olimpio Education,14840.0,7.42,Qf_votes_round,votes_round,olimpioeducation
3,Trustalabs,14840.0,7.42,Qf_no_passport_filtered_votes_round,no_passport_filtered_votes_round,trustalabs
4,Wizard Bridge EVM,14840.0,7.42,Qf_filtered_votes_round,filtered_votes_round,wizardbridgeevm
5,Trustalabs,14840.0,7.42,Qf_eth_model_votes,eth_model_votes,trustalabs
6,Olimpio Education,14840.0,7.42,Qf_no_passport_filtered_votes_round,no_passport_filtered_votes_round,olimpioeducation
7,Hypercerts Foundation,14840.0,7.42,Qf_filtered_votes_round,filtered_votes_round,hypercertsfoundation
8,Metopia,14840.0,7.42,Qf_no_passport_filtered_votes_round,no_passport_filtered_votes_round,metopia
9,Hypercerts Foundation,14840.0,7.42,Qf_no_passport_filtered_votes_round,no_passport_filtered_votes_round,hypercertsfoundation


In [124]:
# Define your groups as lists of strategies
strategies_to_compare = ['Qf_filtered_votes_round', 'Qf_no_passport_filtered_votes_round', 'COCM_filtered_votes_round', 'COCM_no_passport_filtered_votes_round']

# Use the function to filter data and plot graph for each group
result_df, percent_difference_fig = generate_percent_difference_bargraph(matching_df, strategies_to_compare)
percent_difference_fig.show()
result_df.head(10)

IndexError: list index out of range

In [250]:
def calculate_total_absolute_difference(df, strategy1, strategy2):
    # Filter the dataframe to include only the two strategies
    df_filtered = df[df['Strategy'].isin([strategy1, strategy2])]
    
    # Pivot the data to have strategies as columns and projects as rows
    df_pivot = df_filtered.pivot(index='Project', columns='Strategy', values='Matching Amount')
    
    # Calculate the absolute difference for each project
    df_pivot['Absolute_Difference'] = abs(df_pivot[strategy1] - df_pivot[strategy2])
    
    # Sum the absolute differences
    total_absolute_difference = df_pivot['Absolute_Difference'].sum()
    
    return total_absolute_difference

# Example usage
strategy1 = 'Qf_no_passport_filtered_votes_round'
strategy2 = 'Qf_filtered_votes_round'
matching_df = pd.read_csv('data/gg19_Web3 Community and Education_matching_df.csv')
matching_df['Strategy'] = matching_df['Strategy'] + "_" + matching_df['Dataset']

total_abs_diff = calculate_total_absolute_difference(matching_df, strategy1, strategy2)
print(f"The total absolute difference in Matching Amount between '{strategy1}' and '{strategy2}' is: {total_abs_diff}")


The total absolute difference in Matching Amount between 'Qf_no_passport_filtered_votes_round' and 'Qf_filtered_votes_round' is: 9643.456736965965


In [227]:
matching_df['Strategy'].unique()

array(['Qf_filtered_votes_round', 'Qf_eth_model_votes', 'Qf_votes_round',
       'COCM_filtered_votes_round', 'COCM_votes_round',
       'COCM_eth_model_votes'], dtype=object)

In [251]:
from scipy.stats import pearsonr

def calculate_correlation(df, strategy1, strategy2, baseline):
    # Filter the dataframe to include only the required strategies
    df_filtered = df[df['Strategy'].isin([strategy1, strategy2, baseline])]
    
    # Pivot the data to have strategies as columns and projects as rows
    df_pivot = df_filtered.pivot(index='Project', columns='Strategy', values='Matching Amount')
    
    # Calculate the project-level differences
    df_pivot[f'{strategy1}_diff'] = df_pivot[strategy1] - df_pivot[baseline]
    df_pivot[f'{strategy2}_diff'] = df_pivot[strategy2] - df_pivot[baseline]
    
    # Calculate the Pearson correlation coefficient
    corr, _ = pearsonr(df_pivot[f'{strategy1}_diff'], df_pivot[f'{strategy2}_diff'])
    
    return corr

# Example usage
strategy1 = 'COCM_filtered_votes_round'
strategy2 = 'Qf_filtered_votes_round'
baseline = 'Qf_no_passport_filtered_votes_round'

correlation = calculate_correlation(matching_df, strategy1, strategy2, baseline)
print(f"The Pearson correlation coefficient between the project-level differences of '{strategy1}' and '{strategy2}' from the baseline '{baseline}' is: {correlation}")

The Pearson correlation coefficient between the project-level differences of 'COCM_filtered_votes_round' and 'Qf_filtered_votes_round' from the baseline 'Qf_no_passport_filtered_votes_round' is: -0.13025503114664508


In [270]:
import plotly.graph_objects as go

def create_scatter_plot(df, strategy1, strategy2, baseline):
    # Filter the dataframe to include only the required strategies
    df_filtered = df[df['Strategy'].isin([strategy1, strategy2, baseline])]
    
    # Pivot the data to have strategies as columns and projects as rows
    df_pivot = df_filtered.pivot(index='Project', columns='Strategy', values='Matching Amount')
    
    # Calculate the project-level differences
    df_pivot[f'{strategy1}_diff'] = df_pivot[strategy1] - df_pivot[baseline]
    df_pivot[f'{strategy2}_diff'] = df_pivot[strategy2] - df_pivot[baseline]
    
    # Create the scatter plot with x-axis log scale
    fig = go.Figure(data=go.Scatter(
        x=df_pivot[f'{strategy1}_diff'],
        y=df_pivot[f'{strategy2}_diff'],
        mode='markers',
        text=df_pivot.index,
        hoverinfo='text+x+y'
    ))
    
    fig.update_layout(
        title=f"Scatter Plot of Project-Level Differences",
        xaxis_title=f"Difference between {strategy1} and {baseline}",
        yaxis_title=f"Difference between {strategy2} and {baseline}",
        #xaxis_type="log",  # Set x-axis to log scale
        font=dict(size=12),
        height=600,
        width=800
    )
    
    return fig

# Example usage
strategy1 = 'COCM_filtered_votes_round'
strategy2 = 'Qf_eth_model_votes'
baseline = 'Qf_filtered_votes_round'

scatter_plot = create_scatter_plot(matching_df, strategy1, strategy2, baseline)
scatter_plot.show()

,Project,legitimacy_score
0,Wizard Bridge EVM,1
1,Hypercerts Foundation,5
2,Olimpio Education,2
3,Metopia,2
4,ZachXBT,5
...,...,...
305,Empowering Web3 Education with GetCrypto Card ...,3
306,Poly Raiders (Proof of Charity),3
307,Greenpill Kenya,3
308,Learn Self-Custody with irish,3
